## Открытие файла с данными и изучение информации

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier


In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.describe() 

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [5]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

**Вывод:** В данных всего 3214 строк и 5 колонок, с количественными и вещественными типами данных. Числовые данные распределены достаточно равномерно. Столбец "is_ultra" содержит категориальный признак, что означает тип тарифа, «Ультра» — 1, «Смарт» — 0. В данном столбце других значений нет. Датасет готов к исследованию.

## Разделение данных на выборки

In [6]:
df_train, df_test = train_test_split(df, test_size=0.25, random_state=12345)
df_train, df_valid = train_test_split(df_train, test_size=0.333, random_state=12345)

In [7]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)
print(features_valid.shape)
print(target_valid.shape)

(1607, 4)
(1607,)
(804, 4)
(804,)
(803, 4)
(803,)


**Вывод:** Данные разбили на три выборки в соотношении 50%, 25% и 25%. Создали параметр с признаками и целевым признаком.

## Исследование моделей

### Модель: решающее дерево

In [8]:
best_model_decision_tree = None
best_result_decision_tree = 0
for depth in range(1, 6):
	model_decision_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
	model_decision_tree.fit(features_train,target_train) 
	predictions = model_decision_tree.predict(features_valid) 
	result = accuracy_score(target_valid, predictions) 
	if result > best_result_decision_tree:
		best_model_decision_tree = model_decision_tree
		best_result_decision_tree = result
        
print("Accuracy лучшей модели решающие дерево валидационной выборки:", best_result_decision_tree)
print("Лучшее гиперпараметры модели решающие дерево валидационной выборки:", best_model_decision_tree)

Accuracy лучшей модели решающие дерево валидационной выборки: 0.7870485678704857
Лучшее гиперпараметры модели решающие дерево валидационной выборки: DecisionTreeClassifier(max_depth=3, random_state=12345)


### Модель: случайный лес

In [9]:
best_model_random_forest = None
best_result_random_forest = 0
for est in range(1, 52, 10):
    for depth in range(1, 15, 2):
        model_random_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model_random_forest.fit(features_train,target_train ) 
        result = model_random_forest.score(features_valid, target_valid)
        if result > best_result_random_forest:
            best_model_random_forest = model_random_forest
            best_result_random_forest =result 

print("Accuracy наилучшей модели случайный лес на валидационной выборке:", best_result_random_forest)
print("Лучшее гиперпараметры модели случайный лес на валидационной выборки:", best_model_random_forest)

Accuracy наилучшей модели случайный лес на валидационной выборке: 0.8194271481942715
Лучшее гиперпараметры модели случайный лес на валидационной выборки: RandomForestClassifier(max_depth=9, n_estimators=31, random_state=12345)


### Модель: логистической регрессии

In [10]:
best_model_logistic_regression = None
best_result_logistic_regression = 0
for max_iter in range(1000, 3000, 1000):
    for solver in ('newton-cg','lbfgs','liblinear', 'sag', 'saga'):
	    model_logistic_regression = LogisticRegression(random_state=12345, solver=solver, max_iter=max_iter) # обучите модель с заданной глубиной дерева
	    model_logistic_regression.fit(features_train,target_train) # обучите модель
	    predictions = model_logistic_regression.predict(features_valid) # получите предсказания модели
	    result = accuracy_score(target_valid, predictions) # посчитайте качество модели
	    if result > best_result_logistic_regression:
		    best_model_logistic_regression = model_logistic_regression
		    best_result_logistic_regression = result
        
print("Accuracy лучшей модели логистической регрессии валидационной выборки:", best_result_logistic_regression)
print("Лучшее гиперпараметры модели логистической регрессии валидационной выборки:", best_model_logistic_regression)

/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:305: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
/opt/conda/lib/python3.9/site-packages/scipy/optimize/_linesearch.py:456: LineSearchWarning: The line search algorithm did not converge
  warn('The line searc

Accuracy лучшей модели логистической регрессии валидационной выборки: 0.7372353673723536
Лучшее гиперпараметры модели логистической регрессии валидационной выборки: LogisticRegression(max_iter=1000, random_state=12345, solver='newton-cg')


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


**Вывод:** Провели исследование данных на трех моделях, меняя гиперпараметры.</font><br>
В модели "Решающее дерево" меняли один гиперпараметр (глубина дерева), в диапазоне от 1-5. Получили лучшее Accuracy  0.794, при глубине 3.</font><br>
В модели "Случайный лес" меняли два гиперпараметра (количество листьев и глубина дерева) в диапазоне от 1-50, с шагом 10, и 1-10 соответственно. Получили лучшее Accuracy  0.834, при глубине 9 и количестве листьев 21.</font><br>
В модели "Логистическая регрессия" меняли два гиперпараметра (количество итераций обучения и алгоритмы, который будут строить модель). Получили лучшее Accuracy  0.741, при количество итераций обучения 1000 и алгоритме "newton-cg".</font><br>
Соответственно для дальнейшей проверке на тестовой выборке выбираем модель "Случайны лес", т.к. они имеет наилучшее качество.

## Проверка модели на тестовой выборке

In [11]:
model= RandomForestClassifier(random_state=12345, n_estimators=21, max_depth=9)
model.fit(features_train, target_train)
predictions = model.predict(features_test) 
result_random_forest = accuracy_score(target_test, predictions)

        
print("Accuracy лучшей модели на тестовой выборке:", result_random_forest)

Accuracy лучшей модели на тестовой выборке: 0.7960199004975125


**Вывод:** Цель достигнута, Accuracy на тестовой выборке равна 0.8. Модель "Случайный лес" можно применять в бизнесе.

## Проверка модели на адекватность

In [12]:
dummy_clf = DummyClassifier(strategy='most_frequent', random_state=12345)
dummy_clf.fit(features_train, target_train)

dummy_clf.score(features_test, target_test)

0.7002487562189055

**Вывод:** При проверке выбранной модели на адекватность сравним ее с более простым классификатором, DummyClassifier. Правильность данной модели составила 0.7, что подтверждает правильность использования выбранной модели "Случайный лес".